In [1]:
import os

In [2]:
%pwd

'c:\\Users\\yakle\\OneDrive\\Desktop\\End_to_End_ml_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\yakle\\OneDrive\\Desktop\\End_to_End_ml_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_files: Path
    unzip_dir: Path

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_files=config.local_data_files,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [9]:
class DataIngestion:

    def __init__(self,config: DataIngestionConfig):
        self.config =config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_files):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_files
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_files))}")


    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_files, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-02-20 21:18:46,685: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-02-20 21:18:46,685: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-02-20 21:18:46,690: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2024-02-20 21:18:46,690: INFO: common: created directoy at: artifacts]
[2024-02-20 21:18:46,690: INFO: common: created directoy at: artifacts/data_ingestion]
[2024-02-20 21:18:47,996: INFO: 3711438728: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b75e8cfcb3a03dc59cc15b9460eb3735a9c4d12551441de10275e37d002ae808"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F08A:26A4E:FC056:140210:65D4C9DF
Accept-Ranges: bytes
Date: Tue, 20 Feb 20